In [1]:
# Hi, this is an AI agent that generate radiology reports from adult glioma patients MRI and tabular data .

# At frist step importing essentail libraries 
!pip install SimpleITK
!pip install nibabel --quiet


import SimpleITK as sitk
import nibabel as nib
import numpy as np
import pandas as pd 
import tensorflow as tf 

import keras
import os 
import cv2
import glob 
import pathlib as Path
import matplotlib.pyplot as plt
import re 
import tensorflow_datasets as tfds
import random
from keras import regularizers
from keras.models import Model

from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization , Input , Reshape 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import plot_model
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from keras import layers
from collections import defaultdict
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision
import tqdm as tqdm

# By seting on mixed_float16 , reduce the computational cost 
mixed_precision.set_global_policy("mixed_float16")



!nvidia-smi
strategy = tf.distribute.MirroredStrategy(["GPU:0", "GPU:1"])
print(f"Number of GPUs Available: {strategy.num_replicas_in_sync}")

2025-09-03 13:05:32.026829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756904732.230470      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756904732.290033      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Wed Sep  3 13:05:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

I0000 00:00:1756904745.844933      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [2]:



image_data_folders = ["/kaggle/input/glioma-train-test/train data"]
patient_files = defaultdict(list)
for folder in image_data_folders :
    for root,dirs,files in os.walk(folder):
        for file in files :
            if file.endswith('.nii') or file.endswith('.nii.gz'):
                patient_id = '_'.join(file.split('_')[:1])
                patient_path = os.path.join(root,file)
                patient_files[patient_id].append(patient_path)


def load_patient_onehot(patient_id, files_by_patient, labels_df):
    paths = files_by_patient.get(patient_id, [])
    image_modalities = {}
    loader = LoadImage(image_only=True)
def nifti_loader_3D (path, expected_dim = 3 , min_size = (112,112,80)):
    try:
        img = sitk.ReadImage(path)

    except Exception as e :
        raise ValueError (f'cannot read {path}:{e}')

    array = array = sitk.GetArrayFromImage(img)

    if array.ndim != expected_dim :
        raise ValueError (f'unexpected dimention  {array.ndim} in {path}')

    if any(s<m for s , m in zip(array.shape,min_size)):
        raise ValueError (f'unexpected size  {array.shape} in {path}')

    if not np.isfinite(array).all():
        raise ValueError (f'unexpected value  in {path}')
    return img  





def resampler (image , spacing = (2,2,2), origin = None , direction = None ,is_label = False ):
    image_size = image.GetSize()
    image_spacing = image.GetSpacing()
    final_size = [int(np.round(image_size[i]*(image_spacing[i]/spacing[i]))) for i in range(3) ]
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(spacing)
    resampler.SetSize(final_size)
    if direction is None : 
        resampler.SetOutputDirection(image.GetDirection())
    else :
        resampler.SetOutputDirection(direction)
    if origin is None :
        resampler.SetOutputOrigin(image.GetOrigin())
    else :
        resampler.SetOutputOrigin(origin)

    
    resampler.SetTransform(sitk.Transform())

    if is_label : 
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)

    return resampler.Execute(image)

    



def crop_pad(image, target_size=(224, 224, 160 ), pad_value=0):

    image_size = list(image.GetSize())  
    target_size = list(target_size)
    
    start_index = []
    size_after_crop = []
    pad_lower = []
    pad_upper = []
    
    for dim in range(3):
        if image_size[dim] >= target_size[dim]:
            # crop center
            start = (image_size[dim] - target_size[dim]) // 2
            start_index.append(start)
            size_after_crop.append(target_size[dim])
            pad_lower.append(0)
            pad_upper.append(0)
        else:
           
            start_index.append(0)
            size_after_crop.append(image_size[dim])
            total_pad = target_size[dim] - image_size[dim]
            lower = total_pad // 2
            upper = total_pad - lower
            pad_lower.append(lower)
            pad_upper.append(upper)

    
    cropped = sitk.RegionOfInterest(image, size_after_crop, start_index)
    
    padded = sitk.ConstantPad(cropped, pad_lower, pad_upper, pad_value)

    return padded


def rescaling(image):
    p1 = np.percentile(image,1)
    p99 = np.percentile(image,99)
    image = np.clip(image,p1,p99)
    img = (image - np.mean(image)) / (np.std(image) + 1e-8)
    return img



def sitk_to_numpy(image):
    array = sitk.GetArrayFromImage(image)
    array = np.expand_dims(array,axis = 0)
    return array.astype(np.float32)










In [3]:


def load_and_modify_patients(patient_id, patient_files ):
    # --- Load images ---
    paths = patient_files.get(patient_id, [])
    image_modalities = {}
    
    for path in paths:
        if not os.path.exists(path) or os.path.getsize(path) < 1000:
            continue
        file_name = path.lower()
        if 't1c' in file_name:
            image_modalities['t1c'] = nifti_loader_3D(path)
        elif 'flair' in file_name:
            image_modalities['flair'] = nifti_loader_3D(path)

    required_modalities = ['t1c', 'flair']
    if not all(k in image_modalities for k in required_modalities):
        raise ValueError(f"Incomplete data for patient {patient_id}")

    t1c_image = resampler(crop_pad(image_modalities['t1c']), is_label=False)
    t2f_image = resampler(crop_pad(image_modalities['flair']), is_label=False)
    t1c_image = sitk_to_numpy(t1c_image).squeeze(0)
    t2f_image = sitk_to_numpy(t2f_image).squeeze(0)

    if t1c_image.shape != t2f_image.shape:
        raise ValueError(f"Shape mismatch for patient {patient_id}: T1c {t1c_image.shape}, T2f {t2f_image.shape}")

    t1c_image = rescaling(t1c_image)
    t2f_image = rescaling(t2f_image)
    image = np.stack([t1c_image, t2f_image], axis=0)
    image = np.nan_to_num(image)

    if np.max(image) == 0:
        raise ValueError(f"All-zero image for patient {patient_id}")
    if np.std(image) < 1e-6:
        raise ValueError(f"Flat image (no variation) for patient {patient_id}")

    image = np.transpose(image, (3, 2, 1, 0))

    
    
    return image

sorted_patient_ids = sorted(patient_files.keys(), key=lambda x: int(x.split('-')[-1]))

image_list = []
for pid in sorted_patient_ids:
    try:
        img = load_and_modify_patients(pid, patient_files)
        image_list.append(img)
    except Exception as e:
        print(f"Skipping {pid}: {e}")

image_array = np.stack(image_list, axis=0)
print(f"Final image array shape: {image_array.shape}")

Final image array shape: (445, 112, 112, 80, 2)


In [4]:
labels_df = pd.read_csv("/kaggle/input/traincsv2/Traintabular.csv") 


ids = labels_df['ID'].tolist()
sorted_ids = sorted(ids, key=lambda x: int(x.split('-')[-1]))


tabular_list = []
tumor_type_list = []
tumor_grade_list = []
MGMT_list = []
IDH_list = []
survival_list = []


age_mean = labels_df['Age'].mean()
age_std = labels_df['Age'].std()
os_min = labels_df['OS'].min()
os_max = labels_df['OS'].max()


for pid in sorted_ids:
    try:
        row = labels_df[labels_df['ID'] == pid].iloc[0]
        
       
        sex = np.array([row['Sex']], dtype=np.float32)
        age = (row['Age'] - age_mean) / age_std  # standardize
        tabular_input = np.concatenate([sex, [age]], axis=0)
        tabular_list.append(tabular_input)
        
       
        tumor_type_list.append(int(row['Tumor_type']))
        tumor_grade_list.append(int(row['Tumor_grade']))
        MGMT_list.append(int(row['MGMT']))
        IDH_list.append(int(row['IDH']))
        
        
        survival_val = (row['OS'] - os_min) / (os_max - os_min)
        survival_list.append(survival_val)
        
    except Exception as e:
        print(f"Skipping {pid}: {e}")


tabular_array = np.stack(tabular_list, axis=0)
tumor_type_array = np.array(tumor_type_list)
tumor_grade_array = np.array(tumor_grade_list)
MGMT_array = np.array(MGMT_list)
IDH_array = np.array(IDH_list)
survival_array = np.array(survival_list)

tumor_type_onehot = tf.keras.utils.to_categorical(tumor_type_array, num_classes=3).astype(np.float32)
tumor_grade_onehot = tf.keras.utils.to_categorical(tumor_grade_array, num_classes=3).astype(np.float32)
MGMT_onehot       = tf.keras.utils.to_categorical(MGMT_array, num_classes=3).astype(np.float32)
IDH_onehot        = tf.keras.utils.to_categorical(IDH_array, num_classes=2).astype(np.float32)

print("Shapes:")
print("Tabular:", tabular_array.shape)
print("Tumor type:", tumor_type_array.shape)
print("Tumor grade:", tumor_grade_array.shape)
print("MGMT:", MGMT_array.shape)
print("IDH:", IDH_array.shape)
print("Survival:", survival_array.shape)


print("tumor_type_onehot:", tumor_type_onehot.shape)
print("tumor_grade_onehot:", tumor_grade_onehot.shape)
print("MGMT_onehot:", MGMT_onehot.shape)
print("IDH_onehot:", IDH_onehot.shape)
print("survival:", survival_array.shape)

Shapes:
Tabular: (445, 2)
Tumor type: (445,)
Tumor grade: (445,)
MGMT: (445,)
IDH: (445,)
Survival: (445,)
tumor_type_onehot: (445, 3)
tumor_grade_onehot: (445, 3)
MGMT_onehot: (445, 3)
IDH_onehot: (445, 2)
survival: (445,)


In [5]:

images_tensor = tf.convert_to_tensor(image_array, dtype=tf.float32)
tabular_tensor = tf.convert_to_tensor(tabular_array, dtype=tf.float32)
tumor_type_tensor = tf.convert_to_tensor(tumor_type_onehot, dtype=tf.float32)
tumor_grade_tensor = tf.convert_to_tensor(tumor_grade_onehot, dtype=tf.float32)
MGMT_tensor = tf.convert_to_tensor(MGMT_onehot, dtype=tf.float32)
IDH_tensor = tf.convert_to_tensor(IDH_onehot, dtype=tf.float32) 
survival_tensor = tf.convert_to_tensor(survival_array, dtype=tf.float32)

dataset = tf.data.Dataset.from_tensor_slices((
    {"image_input": images_tensor, "tabular_input": tabular_tensor}, 
    {
        "tumor_type": tumor_type_tensor,
        "tumor_grade": tumor_grade_tensor,
        "MGMT": MGMT_tensor,
        "IDH": IDH_tensor,
        "survival": survival_tensor
    }
))


total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size   = total_size - train_size


train_ds = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_ds = remaining.take(val_size)


def augment_3d_mri(img, flip_prob=0.4, noise_prob=0.4, noise_std=0.01):
   

    # --- Random flip along spatial axes ---
    if tf.random.uniform(()) < flip_prob:
        axis = tf.random.uniform([], minval=0, maxval=3, dtype=tf.int32)  # x, y, or z
        img  = tf.reverse(img, axis=[axis])

    # --- Random Gaussian noise ---
    if tf.random.uniform(()) < noise_prob:
        noise = tf.random.normal(shape=tf.shape(img), mean=0.0, stddev=noise_std, dtype=img.dtype)
        img = img + noise

    return img
def augment_fn(x, y):
    x = x.copy()
    x["image_input"] = augment_3d_mri(x["image_input"])
    return x, y


BATCH_SIZE = 3
train_data = (
    train_ds
    .shuffle(buffer_size=len(tabular_array), seed=42)
    .map(augment_fn, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
validation_data = (
    val_ds
    .shuffle(buffer_size=len(tabular_array), seed=42)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
for x, y in train_data.take(1):
    print("Image batch shape:", x["image_input"].shape)
    print("Tabular batch shape:", x["tabular_input"].shape)
    for key, val in y.items():
        print(f"{key}: {val.shape}, dtype={val.dtype}")


Image batch shape: (3, 112, 112, 80, 2)
Tabular batch shape: (3, 2)
tumor_type: (3, 3), dtype=<dtype: 'float32'>
tumor_grade: (3, 3), dtype=<dtype: 'float32'>
MGMT: (3, 3), dtype=<dtype: 'float32'>
IDH: (3, 2), dtype=<dtype: 'float32'>
survival: (3,), dtype=<dtype: 'float32'>


In [6]:

def sensory_neuron (primary_filters, secondry_filters ,kernel_size , dilation , reciver ) :
    a = reciver
    a = layers.Conv3D(primary_filters, kernel_size , padding='same',dilation_rate = dilation, kernel_initializer='he_normal', activation='relu')(a)
    a_2 = layers.Conv3D(secondry_filters , kernel_size, padding='same',dilation_rate = dilation, kernel_initializer='he_normal', activation='relu')(a)
    a_3 = BatchNormalization()(a_2)
    a_4 = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(a_3)
    return a_4



def laterel_geniculate_body (filters ,kernel_size ,dilation, reciver) : 
    f = reciver 
    f_1a = layers.Conv3D(filters , kernel_size , padding='same', dilation_rate = dilation , kernel_initializer='he_normal', activation='relu')(f)
    f_1b = BatchNormalization()(f_1a)
    f_1c = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(f_1b)
    return f_1c



    




def motor_neuron (primary_filters, secondry_filters ,primary_kernel_size,secondry_kernel_size , primary_dilation, secondry_dilation , reciver  ) :

    g = reciver 
    g_4 = layers.Conv3DTranspose(primary_filters, primary_kernel_size, strides=(2, 2, 2), padding="same",dilation_rate = primary_dilation, activation='relu')(g)
    g_4 = layers.Conv3D(primary_filters, primary_kernel_size, padding='same', dilation_rate = primary_dilation ,kernel_initializer='he_normal')(g_4)
    g_4 = BatchNormalization()(g_4)
    g_4x = layers.Conv3DTranspose(secondry_filters, secondry_kernel_size, strides=(2, 2, 2), padding="same",dilation_rate = secondry_dilation, activation='relu')(g)
    g_4x = layers.Conv3D(secondry_filters , secondry_kernel_size , padding='same', dilation_rate = secondry_dilation ,kernel_initializer='he_normal')(g_4x)
    g_4x = BatchNormalization()(g_4x)
    g_5x = layers.concatenate([g_4x,g_4])
    return g_5x






def nucleus (reciver,filters ):
    x = reciver
    x = layers.Conv3D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = layers.Conv3D(filters, 3, padding='same', dilation_rate=(2, 2, 2), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = keras.layers.Add()([ reciver , x])
    x = layers.Activation('relu')(x)
    y = layers.Conv3D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    y = layers.Conv3D(filters, 3, padding='same', dilation_rate=(2, 2, 2),kernel_initializer='he_normal')(y)
    y = BatchNormalization()(y)
    y = keras.layers.Add()([ x , y])
    y = layers.Activation('relu')(y)
    y = layers.Dropout(0.1)(y)
    return y


class spatial_attention (layers.Layer):
     def __init__(self):
         super().__init__()
         self.conv = layers.Conv3D(1, (3,3,3) ,  strides=(1, 1, 1), padding="same",dilation_rate=(2, 2, 2) , activation='sigmoid')
     def call (self,x):
         y = tf.reduce_max(x, axis=-1, keepdims=True)     
         z = tf.reduce_mean(x, axis=-1, keepdims=True)
         w = tf.keras.layers.concatenate([y,z])
         attention = self.conv(w)

         return x* attention


                                    


In [7]:




image_input = layers.Input(shape=(112, 112, 80, 2), name='image_input')

    
nasal = sensory_neuron(32,64,3,(1,1,1),image_input)
nasal_nucleus = nucleus (nasal,64 )
nasal_deeplayerB = spatial_attention()( nasal_nucleus)
    
temporal = sensory_neuron(32,64,5,(1,1,1),image_input)
temporal_nucleus = nucleus (temporal,64)
temporal_deeplayerB = spatial_attention()( temporal_nucleus)
    
crossover = layers.concatenate([nasal_deeplayerB,temporal_deeplayerB]) 
crossover_nucleus = nucleus (crossover,128)
image_featuresA = layers.GlobalMaxPooling3D()(crossover_nucleus)
image_featuresB = layers.GlobalAveragePooling3D()(crossover_nucleus)


tabular_input = layers.Input(shape=(2,), name='tabular_input')
tabular_featuresA = layers.Dense (16, activation='relu')(tabular_input) 
tabular_featuresB = layers.Dense (16, activation='relu')(tabular_featuresA) 


combined = layers.concatenate([image_featuresA,image_featuresB,tabular_featuresB])
combined = layers.Dense(128, activation='relu')(combined)
combined = layers.Dense(64, activation='relu')(combined)

tumor_type_output = layers.Dense(3, activation='softmax', name='tumor_type')(combined)
tumor_grade_output = layers.Dense(3, activation='softmax', name='tumor_grade')(combined)
MGMT_output = layers.Dense(3, activation='softmax', name='MGMT')(combined)
IDH_output = layers.Dense(2, activation='softmax', name='IDH')(combined)
survival_output = layers.Dense(1, activation='sigmoid', name='survival')(combined)


model = Model(inputs=[image_input, tabular_input],
              outputs=[tumor_type_output, tumor_grade_output, MGMT_output, IDH_output, survival_output])





In [8]:
model.load_weights("/kaggle/input/classification/trainingstepfour.weights.h5",skip_mismatch=True)

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss={
        'tumor_type': tf.keras.losses.CategoricalFocalCrossentropy(alpha=[0.2,0.3,0.5], gamma=2.0, from_logits=False, label_smoothing=0.0,axis=-1,
            reduction='sum_over_batch_size',name='categorical_focal_crossentropy'),
        'tumor_grade': tf.keras.losses.CategoricalFocalCrossentropy(alpha=[0.35,0.35,0.3], gamma=2.0, from_logits=False, label_smoothing=0.0,axis=-1,
            reduction='sum_over_batch_size',name='categorical_focal_crossentropy'),
        'MGMT': tf.keras.losses.CategoricalFocalCrossentropy(alpha=[0.3,0.2,0.5], gamma=2.0, from_logits=False, label_smoothing=0.0,axis=-1,
            reduction='sum_over_batch_size',name='categorical_focal_crossentropy'),
        'IDH': tf.keras.losses.CategoricalFocalCrossentropy(alpha=[0.7,0.3], gamma=2.0, from_logits=False, label_smoothing=0.0,axis=-1,
            reduction='sum_over_batch_size',name='categorical_focal_crossentropy'),
        'survival': tf.keras.losses.Huber(delta=1.0,reduction='sum_over_batch_size',name='huber_loss')
    }, loss_weights={   
            "tumor_type": 1,
            "tumor_grade": 3,
            "IDH": 1,
            "MGMT": 2,
            "survival": 1   
        },
    metrics={
        'tumor_type': 'accuracy',
        'tumor_grade': 'accuracy',
        'MGMT': 'accuracy',
        'IDH': 'accuracy',
        'survival': 'mean_absolute_error'
    }
)


history = model.fit(
        train_data,validation_data=validation_data,         
        epochs=40,
        verbose=1)

Epoch 1/40


I0000 00:00:1756904968.771857      70 service.cc:148] XLA service 0x791a580025d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756904968.773690      70 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1756904970.977459      70 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1756904976.111760      70 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756904976.430531      70 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-09-03 13:09:37.529033: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=0,k3=0} for conv (f16[3,64,112,112,80]{4,3,2,1,0}, u8[0]{0}) custom-call(f16[3,32,112,112,80]{4,3,2,1,0}, f16[

118/119 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - IDH_accuracy: 0.9286 - IDH_loss: 0.0195 - MGMT_accuracy: 0.6950 - MGMT_loss: 0.0479 - loss: 0.2897 - survival_loss: 0.0082 - survival_mean_absolute_error: 0.0941 - tumor_grade_accuracy: 0.8591 - tumor_grade_loss: 0.0424 - tumor_type_accuracy: 0.8933 - tumor_type_loss: 0.0392

E0000 00:00:1756905332.170181      70 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756905332.479080      70 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-09-03 13:15:33.572073: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=0,k3=0} for conv (f16[2,64,112,112,80]{4,3,2,1,0}, u8[0]{0}) custom-call(f16[2,32,112,112,80]{4,3,2,1,0}, f16[64,32,5,5,5]{4,3,2,1,0}, f16[64]{0}), window={size=5x5x5 pad=2_2x2_2x2_2}, dim_labels=bf012_oi012->bf012, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues

119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - IDH_accuracy: 0.9286 - IDH_loss: 0.0196 - MGMT_accuracy: 0.6952 - MGMT_loss: 0.0478 - loss: 0.2897 - survival_loss: 0.0082 - survival_mean_absolute_error: 0.0942 - tumor_grade_accuracy: 0.8593 - tumor_grade_loss: 0.0423 - tumor_type_accuracy: 0.8932 - tumor_type_loss: 0.0392

E0000 00:00:1756905388.410276      70 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756905388.729577      70 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-09-03 13:16:29.829026: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=0,k3=0} for conv (f16[3,64,112,112,80]{4,3,2,1,0}, u8[0]{0}) custom-call(f16[3,32,112,112,80]{4,3,2,1,0}, f16[64,32,5,5,5]{4,3,2,1,0}, f16[64]{0}), window={size=5x5x5 pad=2_2x2_2x2_2}, dim_labels=bf012_oi012->bf012, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues

119/119 ━━━━━━━━━━━━━━━━━━━━ 476s 3s/step - IDH_accuracy: 0.9286 - IDH_loss: 0.0196 - MGMT_accuracy: 0.6955 - MGMT_loss: 0.0478 - loss: 0.2897 - survival_loss: 0.0082 - survival_mean_absolute_error: 0.0942 - tumor_grade_accuracy: 0.8594 - tumor_grade_loss: 0.0423 - tumor_type_accuracy: 0.8931 - tumor_type_loss: 0.0393 - val_IDH_accuracy: 0.7640 - val_IDH_loss: 0.0549 - val_MGMT_accuracy: 0.7191 - val_MGMT_loss: 0.0707 - val_loss: 0.5538 - val_survival_loss: 0.0157 - val_survival_mean_absolute_error: 0.1613 - val_tumor_grade_accuracy: 0.7640 - val_tumor_grade_loss: 0.0924 - val_tumor_type_accuracy: 0.7865 - val_tumor_type_loss: 0.0618
Epoch 2/40
119/119 ━━━━━━━━━━━━━━━━━━━━ 300s 3s/step - IDH_accuracy: 0.8473 - IDH_loss: 0.0390 - MGMT_accuracy: 0.7561 - MGMT_loss: 0.0361 - loss: 0.2734 - survival_loss: 0.0092 - survival_mean_absolute_error: 0.1022 - tumor_grade_accuracy: 0.8889 - tumor_grade_loss: 0.0362 - tumor_type_accuracy: 0.8792 - tumor_type_loss: 0.0445 - val_IDH_accuracy: 0.7865 

In [10]:
model.save_weights("/kaggle/working/classtraining.weights.h5")